In [1]:
from scipy import stats

In [13]:
agent_rankings = stats.rankdata([0, 1, 2, 3], method='average')
(2. * (agent_rankings - 1.) / (4 - 1))

array([0.        , 0.66666667, 1.33333333, 2.        ])